# Τεχνητή Νοημοσύνη: Εργαστηριακή Άσκηση 2
---

Ο στόχος της εργασίας είναι η κατασκευή ενός συστήματος προτάσεων (Recommendation System) για ταινίες. Οι προτάσεις αυτές θα πηγάζουν τόσο από τα χαρακτηριστικά της ταινίας όσο και από ορισμένες αξιολογήσεις του κάθε χρήστη.

Στα δεδομένα της άσκησης περιλαμβάνονται ένα αρχείο με το όνομα movies_metadata.csv το όποιο περιέχει τα χαρακτηριστικά κάθε ταινίας όπως θέμα, σκηνοθέτης ηθοποιοί, λέξεις κλειδιά κ.α. από το imdb καθώς και τα αρχεία ratings.csv τα όποια περιέχουν πραγματικές αξιολογήσεις χρηστών, χωρισμένες σε train και σε test.


# Team Description

>Αναστασάκης Ζαχαρίας:
* email :  zaxarisanastasakis@gmail.com

>Αντωνίου Κωνσταντίνος:
* email : kostas.an2016@gmail.com

---
#Κώδικας για κατασκευή περιβάλλοντος εργασίας

##Μόνο για Google Colab

Κώδικας για να γίνει Mount to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [ ]:
#install swi-prolog
!sudo apt-get install software-properties-common
!sudo apt-add-repository ppa:swi-prolog/stable
!sudo apt-get update
!sudo apt-get install swi-prolog
#install pyswip
!pip install pyswip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.96.24.32.14).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
 Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.
 More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/r

# **Μέρος 1: Μελέτη των Metadata, Δημιουργία κόσμου και των βασικών queries.**


In [ ]:
import pandas as pd
from pyswip import Prolog

In [ ]:
#path = '/content/drive/MyDrive/'
path = '/content/drive/MyDrive/hmmy/ai/lab2/'

In [ ]:
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd 
# Διάβασμα του αρχείου 'movie_metadata.csv' 
data = pd.read_csv(path + "movies_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 
data.head()

,Unnamed: 0,budget,genres,homepage,id,plot_keywords,language,original_title,overview,popularity,production_companies,production_countries,release_date,gross,duration,spoken_languages,status,tagline,movie_title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
0,0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,English,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,2009,United States of America,James Cameron,Zoe Saldana,Sigourney Weaver,Stephen Lang
1,1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,English,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2007-05-19,961000000,169,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007,United States of America,Gore Verbinski,Orlando Bloom,Keira Knightley,Stellan Skarsgård
2,2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,Français,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2015-10-26,880674609,148,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,A Plan No One Escapes,Spectre,6.3,4466,2015,United Kingdom,Sam Mendes,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,English,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-07-16,1084939099,165,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,2012,United States of America,Christopher Nolan,Michael Caine,Gary Oldman,Anne Hathaway
4,4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,English,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-07,284139100,132,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,2012,United States of America,Andrew Stanton,Lynn Collins,Samantha Morton,Willem Dafoe


##Μέρος 1α

Παρακάτω δημιουργούμε την συνάρτηση $clean\ text$ η οποία αφαιρεί χαρακτήρες από τα δεδομένα μας, που βρίσκονται στο dataset $movies\_metadata.csv$, όπως τα "κενά" κ.α. για την διευκόληνση της επεξεργασίας τους.

Στην συνέχεια, ορίζουμε τον "κόσμο" της $Prolog$, αφού δημιουργούμε τα κατηγορήματα της και τα αποθηκεύουμε στην δοθείσα λίστα $literals$. Αμέσως μετά την δημιουργία τους, μέσω της εντολής $assertz$ εισάγουμε τα κατηγορήματα δημιουργώντας εν τέλει τον "κόσμο" της $Prolog$. 


In [ ]:
def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  text = text.replace(u'"', u'')
  return text

In [ ]:
#create World
#Ορίζουμε τον κόσμο μας
prolog = Prolog()

#Για κάθε row του πίνακα φτιάχνουμε τα κατηγορήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals
literals = []
plot_keywords = []
moviess = []
black_and_white = set()


for row in data.itertuples(index=True, name='Pandas'):

  movie_title = clean_text(getattr(row, 'movie_title'))
  moviess.append(movie_title)
  director_name = clean_text(getattr(row, 'director_name'))  ####### mporei me thn prolog na mhn ta theloyme se dicts tha doyme meta 
  actor1 = clean_text(getattr(row, 'actor_1_name'))
  actor2 = clean_text(getattr(row, 'actor_2_name'))
  actor3 = clean_text(getattr(row, 'actor_3_name'))
  language = clean_text(getattr(row, 'language'))


  if movie_title != "UNK":

    if director_name != 'UNK':
      literals.append("director('"+ movie_title +"','" + director_name + "')")
    if actor1 != 'UNK':

      literals.append("actor1('"+ movie_title +"','" + actor1 + "')") 
    if actor2 != 'UNK':

      literals.append("actor2('"+ movie_title +"','" + actor2 + "')")
    if actor3 != 'UNK':

      literals.append("actor3('"+ movie_title +"','" + actor3 + "')") 
    if language != 'UNK':
      literals.append("language('"+ movie_title +"','" + language + "')")
          
    for genre in getattr(row, 'genres').split("|"):
      literals.append("genre('"+ movie_title +"','"+ genre +"')")


    for plot_key in getattr(row, 'plot_keywords').split("|"):
      literals.append("plot_keywords('"+ movie_title +"','"+ clean_text(plot_key) +"')")
      if plot_key == 'black and white':

        black_and_white.add(clean_text(getattr(row,'movie_title')))


for item in moviess:
  if item in black_and_white:
    literals.append("color('"+ item +"','"+ '0' +"')")
  else:
    literals.append("color('"+ item +"','"+ '1' +"')") 
    
#Η Prolog θέλει τα κατηγορήματά της με την σειρά 

literals.sort()
for literal in literals:
  prolog.assertz(literal)


##Μέρος 1β

Σε αυτό το μέρος, δημιουργούμε τις ζητούμενες συναρτήσεις οι οποίες βρίσκουν όλες τις ταινίες που έχουν τα παρακάτω χαρακτηριστικά:

1. $common\_genre3\ :$ 3 κοινές λέξεις και άνω σχετικά με το θέμα.  
2. $common\_genre2\ :$ 2 κοινές λέξεις σχετικά με το θέμα.
3. $common\_genre1\ :$ 1 κοινή λέξη σχετικά με το θέμα.
4. $common\_director\ :$ κοινός σκηνοθέτης.
5. $common\_plot4\ :$ 4 και άνω κοινές λέξεις-κλειδιά της πλοκής.
6. $common\_plot1\ :$ 1 έως και 3 κοινές λέξεις-κλειδιά της πλοκής.
7. $common\_actors3\ :$ Ίδιους τους βασικούς ηθοποιούς.
8. $common\_actors2\ :$ 2 από τους 3 βασικούς ηθοποιούς κοινούς.
9. $common\_actors1\ :$ 1 από τους 3 βασικούς ηθοποιούς κοινό.
10. $common\_language\ :$ κοινή γλώσσα.
11. $common\_color\ :$ Ασπρόμαυρη ή έγχρωμη ταινία. 

Οι συναρτήσεις αυτές εμπεριέχονται στο prolog αρχείο που θα κάνουμε colsult στο επόμενο βήμα.

# **2ο Μέρος: Recommendation System με βάση μόνο τα χαρακτηριστικά των ταινιών.**

Στο σημείο αυτό με βάση τους κανόνες που κατασκευάστηκαν στο Μέρος 1 θα κατασκευαστούν κατηγορήματα για την ομοιότητα ταινιών. Παρακάτω δίνεται ένα μικρό παράδειγμα ενός κανόνα και πώς αυτός θα μπορούσε να γραφτεί μέσω του Pyswip. Επίσης όπως αναφέρεται και σε σχόλιο παραπάνω θα μπορεί να γραφτεί και μια βάση δεδομένων με τους κανόνες και να γίνει απευθείας consult.

Στο παρακάτω παράδειγμα το 5 και το 4 εκφράζουν την ομοιότητα των ταινιών π.χ. οι ταινίες που παράγονται μέσω του find_similar_5 είναι πιο όμοιες από αυτές που παράγονται μέσω του find_similar_4. 


In [ ]:
%cd /content/drive/MyDrive/hmmy/ai/lab2    #path for hard_recommendations.pl

/content/drive/MyDrive/hmmy/ai/lab2


## Definition of similar functions

Σε αυτό το μέρος κάνουμε consult το αρχείο $hard\_recommendations.pl$ στο οποίο βρίσκονται οι συνάρτησεις που υλοποιήσαμε στο μέρος 1β καθώς και οι κανόνες ομοιότητας που φτιάξαμε για τον recommender μας. Συγκεκριμένα ορίσαμε τους εξής κανόνες:
- $find\_similar5$ : Για να είναι μία ταινία similar 5 με την ταινία στόχο θα πρέπει να έχει τουλάχιστον 3 genres ίδια καθώς και τουλάχιστον 4 λέξεις κλειδιά της πλοκής να υπάρχουν στην ταινία στόχο.

- $find\_similar4$ : Για να είναι μία ταινία similar 4 με την ταινία στόχο θα πρέπει να έχει τουλάχιστον 2 ηθοποιούς ίδιους ή κοινό σκηνοθέτη καθώς και 2 κοινά genres και απο μία έως τρεις λέξεις κλειδιά κοινές.

- $find\_similar3$ : Για να είναι μία ταινία similar 3 με την ταινία στόχο θα πρέπει να έχει έναν ηθοποιό κοινό, ένα genre κοινό καθώς και απο μία έως τρεις λέξεις κλειδιά κοινές.

- $find\_similar2$ : Για να είναι μία ταινία similar 2 με την ταινία στόχο θα πρέπει να έχει κοινή γλώσσα με αυτήν, ένα genre κοινό καθώς και απο μία έως τρεις λέξεις κλειδιά κοινές.

- $find\_similar1$ : Για να είναι μία ταινία similar 1 με την ταινία στόχο θα πρέπει να έχει κοινή γλώσσα με αυτήν, ένα genre κοινό καθώς και να έχει το ίδιο "χρώμα" (έγχρωμη ή ασπρόμαυρη) με την ταινία στόχο.

$Παρατηρήσεις$ :

- Τα κριτήρια που αναφέρονται παραπάνω παρουσιάζονται με σείρα δυναμικής, δηλαδή το κριτήριο find_similar5 θα δώσει τις ταινίες που είναι πολύ κοντά με την ταινία στόχο, στο κριτήριο find_similar4 τις ταινίες που είναι λιγότερο κοντά κλπ.

- Τα κριτήρια που δημιουργήθηκαν είναι υποκειμενικά καθώς πιστευούμε ότι για να χαρακτηρίσουμε αν μία ταινία πρέπει να προταθεί μεγαλύτερη βαρύτητα έχουν οι κατηγορίες της ταινίας και η πλοκή της και όχι το αν παίζουν σε αυτή οι ίδιοι ηθοποιοί.

In [ ]:
prolog.consult('hard_recommendations.pl')

In [ ]:
#αυτό το cell χρησιμοποιείται μόνο για να τσεκάρουμε αυτόνομα τα κριτήρια που δημιουργήσαμε 
#στο hard_recommendations.pl
q = prolog.query("common_plot4('Men in Black', Y)")

s = set()
lista = []
for soln in q:
    m = soln['Y'] 
    if m not in s:
      s.add(soln['Y'])
print(s)

{'Men in Black II'}


## Simple_Recommender function

In [ ]:
def simple_recommender(movie, part3=False):
    s1 = set()
    s2 = set()
    s3 = set()
    s4 = set()
    s5 = set()
    list_of_movies = []

    q = prolog.query("find_similar5('" + movie +"',M)")
    for soln in q:
        m5 = soln['M'] 
        if m5 not in list_of_movies:
            s5.add(soln['M'])
            list_of_movies.append(soln['M'])
    answers5 = s5

    q = prolog.query("find_similar4('" + movie +"',M)")
    for soln in q:
        m4 = soln['M'] 
        if m4 not in list_of_movies:
            s4.add(soln['M'])
            list_of_movies.append(soln['M'])
    answers4 = s4

    q = prolog.query("find_similar3('" + movie +"',M)")
    for soln in q:
        m3 = soln['M'] 
        if m3 not in list_of_movies:
            s3.add(soln['M'])
            list_of_movies.append(soln['M'])
    answers3 = s3   

    q = prolog.query("find_similar2('" + movie +"',M)")
    for soln in q:
        m2 = soln['M'] 
        if m2 not in list_of_movies:
            s2.add(soln['M'])
            list_of_movies.append(soln['M'])
    answers2 = s2

    q = prolog.query("find_similar1('" + movie +"',M)")
    for soln in q:
      m1 = soln['M']
      if m1 not in list_of_movies:
        s1.add(soln['M'])
        list_of_movies.append(soln['M'])
    answers1 = s1

    q.close()
    if part3 == True:
      return list(answers5), list(answers4), list(answers3), list(answers2), list(answers1)
    else:
      return list(answers5)[:10], list(answers4)[:10], list(answers3)[:10], list(answers2)[:10], list(answers1)[:10]

## Recommendations

In [ ]:
simple_recommender('The Avengers')

(['Captain America: The Winter Soldier',
  'Guardians of the Galaxy',
  'Iron Man',
  'Ant-Man',
  'The Incredible Hulk',
  'The Wolverine',
  'Iron Man 3',
  'Captain America: Civil War',
  'X-Men Origins: Wolverine',
  'Fantastic Four'],
 [],
 ['Scott Pilgrim vs. the World'],
 ['Hellboy II: The Golden Army',
  'Battlefield Earth',
  'Soul Surfer',
  'Knight and Day',
  'The Hitchhikers Guide to the Galaxy',
  'X-Men: The Last Stand',
  '300: Rise of an Empire',
  '300',
  'The Brothers Grimm',
  'The X Files: I Want to Believe'],
 ['Starship Troopers',
  'Kingdom of the Spiders',
  'Doomsday',
  'Hudson Hawk',
  'The Man Who Knew Too Little',
  'RoboCop 3',
  'Spy Kids 2: The Island of Lost Dreams',
  'The Transporter',
  'Bad Company',
  'Sleeper'])

In [ ]:
simple_recommender('Superman')

(['Batman v Superman: Dawn of Justice',
  'Superman IV: The Quest for Peace',
  'Superman III',
  'Batman',
  'Superman Returns',
  'Man of Steel',
  'Superman II'],
 ['Lethal Weapon 3'],
 ['Crimson Tide'],
 ['Hellboy II: The Golden Army',
  'X-Men: The Last Stand',
  'Terminator 3: Rise of the Machines',
  'The Hitchhikers Guide to the Galaxy',
  'Machete Kills',
  'Ant-Man',
  'The Brothers Grimm',
  'Contraband',
  'Austin Powers in Goldmember',
  'The Way of the Gun'],
 ['Kingdom of the Spiders',
  'Doomsday',
  'Hudson Hawk',
  'RoboCop 3',
  'The Good Night',
  'Sliding Doors',
  'Platoon',
  'Thomas and the Magic Railroad',
  'Inception',
  'Half Past Dead'])

In [ ]:
simple_recommender('Harry Potter and the Order of the Phoenix')

(['Harry Potter and the Goblet of Fire',
  'Harry Potter and the Philosophers Stone',
  'Harry Potter and the Chamber of Secrets',
  'Harry Potter and the Prisoner of Azkaban',
  'Harry Potter and the Half-Blood Prince'],
 [],
 [],
 ['Practical Magic',
  'Mission: Impossible II',
  'Star Wars: Episode I - The Phantom Menace',
  'The Witch',
  'Oz: The Great and Powerful',
  'Last Action Hero',
  'Inception',
  'The Sorcerers Apprentice',
  'First Blood',
  'Dune'],
 ['Hellboy II: The Golden Army',
  'Quest for Camelot',
  'Starship Troopers',
  'X-Men: The Last Stand',
  'The Hitchhikers Guide to the Galaxy',
  'The Devils Advocate',
  'The Secret Life of Pets',
  'Hudson Hawk',
  'RoboCop 3',
  '300'])

In [ ]:
simple_recommender('The Hunger Games: Mockingjay - Part 2')

([],
 ['The Hunger Games: Mockingjay - Part 1',
  'The Hunger Games',
  'The Hunger Games: Catching Fire',
  'I Am Legend'],
 ['A Scanner Darkly',
  'Cirque du Freak: The Vampires Assistant',
  'Journey to the Center of the Earth',
  'Journey 2: The Mysterious Island'],
 ['Starship Troopers',
  'Eragon',
  'Mad Max Beyond Thunderdome',
  'Battlefield Earth',
  'Alien³',
  'Terminator 3: Rise of the Machines',
  'Doomsday',
  'Ant-Man',
  '300: Rise of an Empire',
  'RoboCop 3'],
 ['Hellboy II: The Golden Army',
  'Kingdom of the Spiders',
  'Soul Surfer',
  'X-Men: The Last Stand',
  'The Hitchhikers Guide to the Galaxy',
  'The Man Who Knew Too Little',
  'Hudson Hawk',
  'The Transporter',
  'Spy Kids 2: The Island of Lost Dreams',
  '300'])

In [ ]:
simple_recommender('Terminator 3: Rise of the Machines')

(['The Matrix Revolutions',
  'Terminator Genisys',
  'Terminator Salvation',
  'The Matrix Reloaded',
  'The Terminator'],
 ['Surrogates'],
 [],
 ['Starship Troopers',
  'Mad Max Beyond Thunderdome',
  'Battlefield Earth',
  'Alien³',
  'Doomsday',
  'RoboCop 3',
  'Sleeper',
  'Austin Powers in Goldmember',
  '2001: A Space Odyssey',
  'Collateral Damage'],
 ['Kingdom of the Spiders',
  'The Devils Advocate',
  'Run, Hide, Die',
  'Hudson Hawk',
  'Platoon',
  'Dracula 2000',
  'After.Life',
  'Thomas and the Magic Railroad',
  'Inception',
  'Half Past Dead'])

$Παρατηρήσεις$ :

- Σε γενικές γραμμές τα recommendations που παίρνουμε εκ πρώτης όψεος μοιάζουν αρκετά καλά καθώς για κάθε μία απο τις παραπάνω ταινίες στόχους σαν similar 5 ταινίες προτείνονται είτε τα sequels αυτών είτε ταίνιες με αρκετά παρόμοια θεματική ενότητα.

- Παρατηρούμε ότι σε κάποιες περιπτώσεις υπάρχουν πολλές ταινίες στο similar 5 και καμία ταινία στο similar 4. Αυτό είναι λογικό καθώς ταινίες όπως το $The\ Avengers$ έχουν κοινή θεματική καθώς και κοινή πλοκή με αρκετές ταινίες της Marvel οι οποίες και προτείνονται στην κατηγορία similar 5. Όμως επειδή προτείνουμε αυτές τις ταινίες στην παραπάνω κατηγορία δεν μας μένουν άλλες ταινίες οι οποίες να πληρούν τα κριτηρία, ώστε να ανήκουν στην κατηγορία similar 4. Έτσι τονίζεται και πάλι ότι η υποκειμενικότητα στην κατασκευή των κριτηρίων κάνει τα αποτελέσματα ασταθή απο ομάδα σε ομάδα. 

- Ακολουθώντας την ίδια λογική με παραπάνω για την ταινία $The\ Hunger\ Games: Mockingjay - Part 2$ δεν μπορούμε να προτείνουμε κάποια ταινία που θα πληρεί τα κριτήρια που απαιτούνται για την ένταξη της στην κατηγορία similar 5. Όμως βλέπουμε ότι παρ' όλο που μπορεί να μην βρίσκουμε αρκετές ίδιες λέξεις κλειδία στην πλοκή ίδιες με τις υπόλοιπες ταινίες $The\ Hunger\ Games$ καταφέρνουμε να εντάξουμε τις ταινίες αυτές στην κατηγορία similar 4. 

- Γενικότερα το dataset μας περιέχει πολλά sequel πράγμα το οποίο σημαίνει ότι σε αρκετές περιπτώσεις είναι πιθανή η επικάλυψη της κατηγορίας similar 5 με την κατηγορία similar 4 με αποτέλεσμα κάποια απο τις 2 κατηγορίες να μένει κενή. Η συγκεκριμένη συμπεριφορά δεν αποτελεί πρόβλημα καθώς για την συμπλήρωση των κατηγοριών τα κριτήρια ακολουθούνται με την σειρά που θεωρήσαμε εμείς ότι είναι η πιο "σωστή".

# **3ο Μέρος: Recommendation System Με βάση τις προτιμήσεις-Αξιολογήσεις του χρήστη-Εκπαίδευση και Πρόβλεψη**







In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score


rating_weights = {0: -1, 1: -0.5, 2:0, 3:0, 4:0.5, 5:1}
score_weights = {1:1, 2:2, 3:3, 4:4, 5:5} # ανάλογα με τα επίπεδα ομοιότητας που έχουν οριστεί στην simple_recommender

def train_recommender(ratings, rating_weights, score_weights, start=0, end = -1, movie_score = {}, part3 = False):
    """
    Στην συνάρτηση αυτή μπορούμε να ορίζουμε ποιο υποσύνολο των αξιολογήσεων θα χρησιμοποιήσουμε για το train,
    ενώ παράλληλα μπορούμε να αρχικοποιήσουμε και το movie_score το οποίο αποθηκεύονται τα score κάθε ταινίας.
    Έτσι αν έχουμε κάνει train τον recommender μας για τις πρώτες 10 ταινίες, μπορούμε να συνεχίσουμε για τις 
    υπόλοιπες χωρίς κάθε φορά να ξανά-διαμορφώνουμε το score για αυτές. Αυτό θα σας βοηθήσει στην εκτέλεση και χρονικά όταν ελέγχετε την 
    απόδοση σε υποσύνολα των ratings π.χ. πρώτες 10 ταινίες έπειτα τις επόμενες 20 κ.ο.κ.
    """
    if end == -1:
        end = len(ratings)

    ratings = ratings.iloc[range(start, end)]
    for row in tqdm(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        similar_movies = simple_recommender(movie, part3)


        for similar_movie in similar_movies[0]: ### gia tis tainies sto find_similar5
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[5]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[5] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος

        for similar_movie in similar_movies[1]: ### gia tis tainies sto find_similar4
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[4]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[4]    

        for similar_movie in similar_movies[2]: ### gia tis tainies sto find_similar3
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[3]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[3]

        for similar_movie in similar_movies[3]: ### gia tis tainies sto find_similar2
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[2]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[2]

        for similar_movie in similar_movies[4]: ### gia tis tainies sto find_similar1
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[1]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[1]                            

    return movie_score


#αυτό είναι ένα παράδειγμα για το πως θα μπορούσε να υλοποιήθει η predict
#έχουμε ορίσει ότι μια ταινία θα έπρεπε να είναι προτεινόμενη αν είχε σκορ > 0

#στην δικία μας περίπτωση θέλουμε η ταινία να είναι προτεινόμενη αν έχει score>3 
#και επομένως προσαρμόσαμε κατάλληλα την ακόλουθη συνάρτηση
def predict_example(ratings, movie_score):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score: #αν έχουμε σχηματίσει βαθμολογία για την ταινία αυτή
            pred.append(int(movie_score[movie] > 3)) #heuristic για το αν μια ταινία είναι προτεινόμενη
            real.append(int(rating > 3))# έτσι ορίζουμε ότι μια ταινία θα έπρεπε να είναι προτεινόμενη
            #η συνθήκη αυτή δεν μπορεί να αλλάξει
        else: #δεν μπορούμε να προτείνουμε κάτι για το όποιο δεν έχουμε σχηματίσει εικόνα
            pred.append(0)
            real.append(int(rating > 3))
    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics

In [ ]:
train_ratings = pd.read_csv(path + "train_ratings.csv")
test_ratings = pd.read_csv(path + "test_ratings.csv")

## Train and predict using hard_recommendations.pl on a subset

Έχοντας κάνει consult το $hard\_recommendation.pl$ παρατηρήσαμε ότι δεν μπορούσαμε να κάνουμε train σε ολόκληρο το εύρος του training set καθώς το google colab στο οποίο εργαστήκαμε πραγματοποιούσε συνεχώς επανεκκίνηση του πυρήνα δίνοντας το error : $Too\ many\ stacked\ strings$.

Για αυτό το λόγο και για να είναι όσο το δυνατόν καλύτερα τα αποτελέσματα που θα παρουσιάσουμε θα εργαστούμε ως εξής:
- Θα εκπαιδεύσουμε το μοντέλο μας χρησιμοποιώντας το hard_recommendations.pl σε όσο το δυνατόν μεγαλύτερο ποσοστό επί του αρχικού μεγέθους του training set. Η διαδικασία αυτή θα γίνει σταδιακά, ώστε να βλέπουμε τις τιμές των μετρικών σε κάθε βήμα.

- Έπειτα θα δημιουργήσουμε ένα ακομά prolog αρχείο με όνομα $soft\_recommendations.pl$ με το οποίο θα τρέξουμε την εκπαίδευση σε όλο το μήκος του training set. Θα αναφέρουμε περισσότερα στην επόμενη ενότητα.

In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 0, 10, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.47619047619047616, 'recall': 0.1388888888888889, 'f1': 0.2150537634408602}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10, 20, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.48333333333333334, 'recall': 0.4027777777777778, 'f1': 0.43939393939393945}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 20, 30, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.525, 'recall': 0.5833333333333334, 'f1': 0.5526315789473685}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 30, 50, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.5288461538461539, 'recall': 0.7638888888888888, 'f1': 0.6250000000000001}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 50,60, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.5238095238095238, 'recall': 0.7638888888888888, 'f1': 0.6214689265536724}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 75,-1, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.5357142857142857, 'recall': 0.8333333333333334, 'f1': 0.6521739130434783}


Παρατηρούμε ότι καταφέραμε να κάνουμε train πάνω σε 85 δείγματα απο τα 101 που υπάρχουν συνολικά χωρίς να μας κάνει restart το google colab. Οι τελικές τιμές για τις μετρικές που πήραμε είναι:
- $precision = 0.5357$
- $recall = 0.8333$
- $f1\_score = 0.6521$

Το γεγονός ότι δεν καταφέραμε να τελειώσουμε την εκπαίδευση μας κάνει να μην μπορούμε να πούμε με ασφάλεια ότι τα scores που πήραμε είναι τα τελικά καθώς λόγω του μικρού μήκους του dataset μας η τιμές των μετρικών (και συγκεκριμένα του recall) είναι αρκετά ευαίσθητες. Κρίνοντας απο την σταδιακή πορεία της εκπαίδευσης θα περιμέναμε κάποια αισθητή αύξηση του recall και πιθανόν μία μικρή αύξηση του precision.

## Train and predict using solf_recommendations.pl



### Definition of similar functions

Όπως παρατηρήσαμε προηγουμένως χρησιμοποιώντας το $hard\_recommendations.pl$ δεν μπορούσαμε να κάνουμε train σε ολόκληρο το εύρος του training set. Για αυτό το λόγο δημιουργούμε ένα νέο αρχείο κώδικα το $soft\_recommendations.pl$ το οποίο περιέχει τις ίδιες συναρτήσεις με το $hard\_recommendations.pl$ με την μόνη διαφορά στην συνάρτηση $find\_similar5$ η οποία αντί να περιέχει 4 και άνω λέξεις-κλειδιά κοινές από την πλοκή, περιέχει 1 έως 3 κοινές λέξεις-κλειδιά από την πλοκή. Αυτή η αλλαγή ήταν αρκετή ώστε να μας επιτρέψει να εκτελέσουμε την διαδικασία του training πάνω σε όλο το set.


$$Important$$

Για να τρέξετε το παρακάτω κομμάτι κώδικα για την εκπαίδευση με το soft_recommendations.pl πρέπει να γίνει hard reset του kernel στο google colab αλλίως τα αποτελέσματα δεν είναι σωστά. Αυτή ηταν η διαδικασία που ακολουθήσαμε εμείς για να είμαστε σίγουροι για τα αποτελέσματα. 

Επομένως πρέπει να ξανα τρεχτούν όλα τα παραπάνω κελία εκτός του consult που αφορά το hard_recommendations.pl.

In [ ]:
prolog.consult('soft_recommendations.pl')

### Training

In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 0, 10, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.46153846153846156, 'recall': 0.08333333333333333, 'f1': 0.1411764705882353}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10, 20, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.5, 'recall': 0.16666666666666666, 'f1': 0.25}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 20, 30, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.48936170212765956, 'recall': 0.3194444444444444, 'f1': 0.3865546218487395}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 30, 50, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.5360824742268041, 'recall': 0.7222222222222222, 'f1': 0.6153846153846153}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights, 50, 90, movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.5229357798165137, 'recall': 0.7916666666666666, 'f1': 0.6298342541436462}


In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, score_weights,start = 90, movie_score = movie_score, part3 = True)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


{'precision': 0.5267857142857143, 'recall': 0.8194444444444444, 'f1': 0.641304347826087}


## Result analysis

Οι τελικές τιμές για τις μετρικές που πήραμε κατά την εκπαίδευση με το αρχείο solf_recommendations.pl σε όλο το training set είναι:
- $precision = 0.5267$
- $recall = 0.8194$
- $f1\_score = 0.6413$

Όπως περιμέναμε οι τιμές για τις μετρικές με την χρήση του solf_recommendations είναι χειρότερες από αυτές που πήραμε με την χρήση του αρχείου hard_recommendations.pl.

Έτσι, λοιπόν, συμπεραίνουμε ότι η ποιότητα των recommendations ήταν καλύτερη στην πρώτη περίπτωση και παρ' όλη την μη ολοκλήρωση της εκπαίδευσης τα αποτελέσματα που πήραμε ήταν καλύτερα. Αυτά τα παραθέτουμε ξανά παρακάτω:

- $precision = 0.5357$
- $recall = 0.8333$
- $f1\_score = 0.6521$
